In [4]:
# !pip install opencv-python
# !pip install --upgrade pandas
# !pip install --upgrade scipy==1.0.0
# !pip install --upgrade matplotlib==2.1.2
# !pip install --upgrade pyarrow==0.8.0
# ! pip install --upgrade bcolz
# %cd fastai
# !python setup.py install
# !wget http://files.fast.ai/models/weights.tgz /output/weights.tgz
# !tar -xvzf /output/weights.tgz -C /usr/local/lib/python3.6/site-packages/fastai-0.6-py3.6.egg/fastai/ 
# %cp -r /prev_output/data /output/data

weights/
weights/wrn_50_2f.pth
weights/resnext_101_32x4d.pth
weights/resnext_101_64x4d.pth
weights/inceptionresnetv2-d579a627.pth
weights/inceptionv4-97ef9c30.pth
weights/resnext_50_32x4d.pth


In [2]:
!ls /prev_output

Untitled.ipynb	data  fastai  run_cpu.sh  weights.tgz


In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *


In [3]:
PATH = "/output/data/"
arch=resnext101_64
sz=224
bs = 64

In [4]:
!ls {PATH}

data  labels.csv  raw  sample  sample_submission.csv  test


In [5]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_paths(PATH+'raw', tfms=tfms_from_model(arch, sz), trn_name='trn', test_name=PATH+'test/test_raw')


In [6]:
len(data.trn_ds), len(data.test_ds)

(9403, 10357)

In [7]:
len(data.classes), data.classes[:5]

(120,
 ['affenpinscher',
  'afghan_hound',
  'african_hunting_dog',
  'airedale',
  'american_staffordshire_terrier'])

In [8]:
def get_data(sz, bs):
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
    data = ImageClassifierData.from_paths(PATH+'raw', tfms=tfms_from_model(arch, sz), trn_name='trn', test_name=PATH+'test/test_raw')
    return data if sz>300 else data.resize(340, 'tmp')

In [9]:
data = get_data(sz, bs)

A Jupyter Widget

In [10]:
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [14]:
# learn.load('/prev_output/244_pre')

In [15]:
learn.fit(1e-2, 5)

A Jupyter Widget

epoch      trn_loss   val_loss   accuracy                    
    0      0.880837   0.344007   0.93446   
    1      0.400692   0.258726   0.945277                    
    2      0.292783   0.226086   0.942873                    
    3      0.236573   0.215433   0.948883                    
    4      0.187454   0.2145     0.940469                    



[0.2144995, 0.94046945755298317]

In [16]:
learn.precompute=False
learn.fit(1e-2, 5, cycle_len=1)

A Jupyter Widget

epoch      trn_loss   val_loss   accuracy                    
    0      0.159908   0.198925   0.952489  
    1      0.147165   0.199499   0.954893                    
    2      0.131558   0.197877   0.951287                    
    3      0.120718   0.196745   0.948883                    
    4      0.111307   0.198633   0.948883                    



[0.19863334, 0.94888291909144473]

In [19]:
log_preds, y = learn.TTA(is_test=True)

In [20]:
probs = np.exp(log_preds)

In [21]:
probs.shape

(5, 10357, 120)

In [23]:
probs_sub = np.mean(probs, axis=0)

In [25]:
probs_sub.shape

(10357, 120)

In [26]:
ds = pd.DataFrame(probs_sub)
ds.columns = data.classes

In [32]:
ds.insert(0, 'id', [o[17:-4] for o in data.test_ds.fnames])

In [33]:
ds.head()


,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,84a96a551d15bf720c493f47761af4ea,3.808013e-06,3.249541e-05,4.665768e-06,1.737382e-06,1.382965e-06,5.557417e-06,2.745750e-06,2.887320e-07,5.424868e-05,...,1.069401e-05,3.262106e-06,1.572971e-06,1.853882e-06,0.000006,3.834589e-04,0.000012,1.314766e-06,0.000001,2.460548e-05
1,26ae54652bee66cb5a6889f047d60fe2,3.937307e-05,1.355000e-04,9.735779e-05,3.005101e-04,3.381206e-05,1.152910e-04,1.211461e-03,1.166265e-04,3.446757e-05,...,1.437897e-05,3.370773e-04,7.493365e-04,1.005046e-04,0.000040,3.839174e-05,0.000050,3.602639e-04,0.019042,2.152422e-04
2,18d344e127337b8acc18e7dfd84b49f8,5.950593e-08,7.092345e-07,2.050035e-07,2.183114e-07,4.323193e-07,2.284433e-07,3.413004e-07,6.064209e-08,6.162327e-08,...,3.583424e-07,6.845633e-07,7.560491e-07,4.049818e-08,0.000002,7.595943e-08,0.000022,2.026988e-07,0.000156,3.154424e-07
3,74d03f2ccb344ba6bdeee78d8a5412b2,8.424651e-04,7.446042e-06,2.663242e-05,1.600885e-05,9.682376e-06,9.699923e-05,5.959086e-05,1.673325e-05,5.899946e-06,...,2.138369e-05,4.187243e-05,1.548168e-05,4.674935e-06,0.000002,4.985465e-05,0.000001,2.655264e-06,0.000010,3.937743e-05
4,e0295b9b1bee725d52d74f9af84092cf,3.689341e-07,6.400718e-07,3.937727e-07,8.238379e-06,3.355519e-06,4.492785e-06,1.048439e-06,5.437480e-07,7.745845e-07,...,2.495320e-08,4.402960e-07,8.335172e-06,1.032267e-05,0.000013,3.239259e-07,0.000005,5.885601e-07,0.000003,3.153227e-07


In [34]:
ds.to_csv('sumb.gz', compression='gzip', index=False)

In [36]:
FileLink('sumb.gz')

/output/sumb.gz

In [16]:
learn.save('/output/244_pre')

In [19]:
test = learn.predict(data.test_dl)

KeyboardInterrupt: 

In [20]:
log_preds = learn.predict()

In [22]:
log_preds.shape

(819, 120)

In [23]:
log_preds_test = learn.predict_dl(data.test_dl)

In [24]:
log_preds_test.shape

(10357, 120)

In [31]:
import pickle as pkl
pkl.dump(log_preds_test, open('/output/result.pkl', 'wb'))

In [29]:
!ls /output/

244_pre.h5  fastai		    pg.ipynb	run_gpu.sh
data	    floyd_requirements.txt  run_cpu.sh


In [32]:
def do_clip(arr, mx): return np.clip(arr, (1-mx)/9, mx)


In [ ]:
import keras
keras.metrics.categorical_crossentropy(data.val_y, do_clip(log_preds, 0.93)).eval()


In [35]:
!pip install tensorflow

    100% |████████████████████████████████| 44.4MB 34kB/s  eta 0:00:01
    100% |████████████████████████████████| 3.0MB 501kB/s eta 0:00:01
    100% |████████████████████████████████| 6.4MB 239kB/s eta 0:00:01
    100% |████████████████████████████████| 81kB 10.8MB/s ta 0:00:01
    100% |████████████████████████████████| 890kB 1.6MB/s eta 0:00:01
    100% |████████████████████████████████| 327kB 3.8MB/s eta 0:00:01
    100% |████████████████████████████████| 81kB 10.7MB/s ta 0:00:01
  Running setup.py bdist_wheel for absl-py ... done
  Stored in directory: /root/.cache/pip/wheels/04/f5/7c/5d4eab10ddf87dec875016e74ba289d87270a90fb2662a76fc
  Running setup.py bdist_wheel for futures ... done
  Stored in directory: /root/.cache/pip/wheels/ad/79/48/b32521764d59b16fd1bc0ffd5862f6d3bf770c7d73ea1fb12a
  Running setup.py bdist_wheel for html5lib ... done
  Stored in directory: /root/.cache/pip/wheels/6f/85/6c/56b8e1292c6214c4eb73b9dda50f53e8e977bf65989373c962
Successfully built absl-py future